## Installing/Importing Libraries and Dependencies


In [3]:
!pip3 install librosa

  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320723 sha256=808458beef2abd09cf6a4df81fd04eb1c22a1b4738ec2f712c844b5b0b1a891e
  Stored in directory: c:\users\saniy\appdata\local\pip\cache\wheels\6f\d1\5d\f13da53b1dcbc2624ff548456c9ffb526c914f53c12c318bb4
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23145 sha256=be13342f9fafe1ad67b629f12af7de996e1e4db27090344400d0bd364422afb7
  Stored in directory: c:\users\saniy\appdata\local\pip\cache\wheels\49\5a\e4\df590783499a992a88de6c0898991d1167453a3196d0d1eeb7
Successfully built resampy audioread


In [5]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

## Extracting the feature set

Creating the function of extract_feature to extract the values of 

- mfcc (Mel Frequency Cepstral Coefficient)
- chroma (Chroma pretains to 12 different frequencies)
- mel (Mel Spectogram Frequency)

In [6]:
#Function to extract mfcc, chroma and mel features from a sound file.
# mfcc: Mel Frequency Cepstral Coefficient
#chroma: Pertains to 12 different pitches
#mel: Mel Spectogram Frequency

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result,chroma))

    if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
            
    return result

Creating the dictionary to hold the numbers and the emotions available in the dataset. Emotions that we will be defining in our this mini project are 

- neutral
- calm
- happy
- sad
- angry
- fearful
- disgust
- surprised

In [7]:
# Dictionary to define dataset for emotions

emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

observed_emotions=['calm','happy','fearful','disgust']

After defining the emotions, now we will extract the features and will try to do unix styled pattern matching

In [8]:
#Load and extract the features from each sound file
#using glob library to use unix styled pattern matching
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("audio_samples\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size,random_state=9)


## Splitting the data in Train and Test Datasets

In [14]:
#split the dataset
x_train, x_test, y_train, y_test=load_data(test_size=0.25)

In [15]:
#Get the shape of the training and test datasets
print((x_train.shape[0],x_test.shape[0]))

(576, 192)


In [16]:
#x gives us regressor/feature values,let's try to extract features
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


## Training the Data

MLP Classifier will be used to train the model. It is a neural network models that trains using backpropagation method. It optimizes the log-loss function using LBFGS or stochastic gradient descent.  

In [21]:
_#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [25]:
#Train the model
model.fit(x_train, y_train)
MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9, beta_2=0.999, early_stopping=False, 
              epsilon=1e-08, hidden_layer_sizes=(300), learning_rate='adaptive',learning_rate_init=0.001, 
              max_iter=500, momentum=True, power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False, warm_start=False)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=300,
              learning_rate='adaptive', max_iter=500, momentum=True)

After extracting the feature/x variables, now we will predict the target/y values, followed by accuracy prediction.

In [30]:
#predicting the y values
y_pred=model.predict(x_test)

## Calculating the Accuracy

In [31]:
#Calculate the accuracy of the model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 76.04%


The accuracy rate the we are achieving is above 76%.